In [16]:
import pandas as pd
import numpy as np
import time
import re

In [13]:
# Methods

def reformat_building_number(bldg):
    if bldg.count(' -') == 1:
        return bldg.replace(' -','-')
    if bldg == '12-Nov':
        return '11-12'
    if bldg == '10-Sep':
        return '9-10'
    if bldg == '10-Feb':
        return '2-10'
    if bldg == '  18-20':
        return '18-20'
    if bldg == '37 A 37C':
        return '37A-37C'
    if bldg == '4-Feb':
        return '2-4'
    if bldg == '12-Aug':
        return '8-12'
    if bldg == '14-Dec':
        return '12-14'
    if bldg == '7-Jan':
        return '1-7'
    if bldg == '10-Feb':
        return '2-10'
    if bldg == '7-Jan':
        return '1-7'
    if bldg == '7-May':
        return '5-7'
    if bldg == '10-Aug':
        return '8-10'
    if bldg == '3-Jan':
        return '1-3'
    if bldg == '12-Oct':
        return '10-12'
    if bldg == '5-Mar':
        return '3-5'
    if bldg == '9-Jul':
        return '7-9'
    if bldg == '4-Mar':
        return '3-4'
    if bldg == '11-Sep':
        return '9-11'
    if bldg == '2-Jan':
        return '1-2'
    if bldg.count(' ') == 1:
        return bldg.replace(' ','-')
    return bldg

def get_LAT_LONG(unit):
    
    # Combine the columns representing the Address part of the input data for geocoding.
    address = str(unit['ST_NUM'])+" " +unit['ST_NAME']+" " +unit['ST_NAME_SUF']+", Boston, MA, " + str(unit['ZIPCODE'])
    # Strip duplicate whitespace
    address = re.sub(r'\s+', ' ',address).strip()
    # print(address)
    
    match_fnd = geocode_df[geocode_df['ADDRESS'].str.match(address)]
    try:
        lat_long = match_fnd.iloc[0].LAT_LONG
        return lat_long
    except:
        return ''
    
#     id_match = geocode_df.ADDRESS.eq(address).idxmax()
#     try:
#         lat_long = geocode_df.iloc[id_match].LAT_LONG
#         return lat_long
#     except:
#         return ''
    
#     lat_long = str(geocode_df[geocode_df['ADDRESS'].str.match(address)].LAT_LONG)
#     if lat_long:
#         ind = lat_long.find(',')
#         print('{} & {}'.format(lat_long[0:ind], lat_long[ind+1:]))
#         if geoType == 'LAT':
#             return lat_long[0:ind]
#         else:
#             return lat_long[ind+1:]        
#     else:
#         return np.nan

In [3]:
# Load address book
col_Names=["ADDRESS", "LAT_LONG"]
geocode_df = pd.read_csv('AddressBook\geocodes.csv', names=col_Names, usecols=[1,5])

geocode_df.head()

,ADDRESS,LAT_LONG
0,"21 CHARLES ST, Boston, MA, 02129","-71.071144,42.38272"
1,"223 LONDON ST, Boston, MA, 02128","-71.03656,42.37541"
2,"23 CHARLES ST, Boston, MA, 02129","-71.07113,42.38272"
3,"125 CONDOR ST, Boston, MA, 02128","-71.03534,42.38259"
4,"211 LONDON ST, Boston, MA, 02128","-71.03667,42.37532"


In [4]:
# Load main dataframe
houses_df = pd.read_csv('housing_cleaned_v3.csv')

# Modify ST_NUM
houses_df['ST_NUM'] = houses_df.apply(lambda row: reformat_building_number(row['ST_NUM']), axis=1)

# FIx ST_SUF
houses_df['ST_NAME_SUF'] = houses_df.apply(lambda row: '' if isinstance(row['ST_NAME_SUF'], float) else row['ST_NAME_SUF'] , axis=1)

# Fix ZIP
houses_df['ZIPCODE'] = houses_df.apply(lambda r: format(int(r['ZIPCODE']), '05'), axis=1)

In [ ]:
t = time.time()
houses_df['LAT_LONG'] = houses_df.apply(lambda r: get_LAT_LONG(r), axis=1)
elapsed = time.time() - t

In [41]:
# houses_df.to_csv('houses_lat_long.csv', index = False)

In [14]:
houses_df_sub = houses_df.tail()
houses_df_sub['LAT_LONG'] = houses_df_sub.apply(lambda r: get_LAT_LONG(r), axis=1)
houses_df_sub

c:\users\sriram\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PID,ST_NUM,ST_NAME,ST_NAME_SUF,ZIPCODE,OWN_OCC,AV_LAND,AV_BLDG,AV_TOTAL,GROSS_TAX,...,BTH_STYLE2,BTH_STYLE3,KITCH_STYLE,HEAT_TYP,AC,FPLACE,INT_CND,INT_FIN,VIEW,LAT_LONG
125744,2205629004,80-82,KENRICK,ST,02135,Y,0,500300,500300,527316,...,NaN,NaN,S,F,N,0.0,A,N,G,
125745,2205642002,12,KENRICK,ST,02135,Y,0,456700,456700,481362,...,NaN,NaN,S,F,C,1.0,A,N,A,
125746,2205642004,12,KENRICK,ST,02135,N,0,439600,439600,463338,...,NaN,NaN,S,W,N,1.0,A,N,A,
125747,2205665002,14,LAKE,ST,02135,N,0,479400,479400,505288,...,NaN,NaN,S,W,N,1.0,A,N,A,
125748,2205665004,12,LAKE,ST,02135,N,0,462200,462200,487159,...,NaN,NaN,S,W,N,1.0,A,N,A,


In [54]:
print('Total {} and no lat long for {}'.format(len(houses_df) , len(houses_df[pd.isnull(houses_df['LAT_LONG'])])))

Total 125749 and not lat long for 4569


In [38]:
houses_df_sub = houses_df[1:10]
houses_df_sub['LAT_LONG'] = houses_df_sub.apply(lambda r: get_LAT_LONG(r), axis=1)
houses_df_sub['LAT_LONG']

c:\users\sriram\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1    -71.03256,42.379173
2     -71.03255,42.37957
3    -71.03248,42.379593
4    -71.03242,42.379616
5    -71.03235,42.379642
6    -71.03229,42.379665
7     -71.03216,42.37971
8    -71.032036,42.37976
9    -71.03197,42.379784
Name: LAT_LONG, dtype: object

In [4]:
address_notes = ['AddressBook\GeocodeResults_1.csv', 
                 'AddressBook\GeocodeResults_2.csv',
                 'AddressBook\GeocodeResults_3.csv', 
                 'AddressBook\GeocodeResults_4.csv',
                 'AddressBook\GeocodeResults_5.csv', 
                 'AddressBook\GeocodeResults_6.csv',
                 'AddressBook\GeocodeResults_snum.csv'
                ]
names=["ADDRESS", "STS_MATCH", "STS_EXACT", "LAT_LONG"]
for i in range(0,len(address_notes)):
    df = pd.read_csv(address_notes[i]);
    df = df
    if i == 0:
        address_bk = df
    else:
        address_bk = pd.concat([address_bk, df])


,17288,"8 COLUMBIA RD, Boston, MA, 02121",Match,Exact,"8 COLUMBIA RD, BOSTON, MA, 02121","-71.08472,42.303516",85728832,L,25,025,090100,1012
0,17289,"39 COLUMBIA RD, Boston, MA, 02121",Match,Non_Exact,"39 COLUMBIA PL, BOSTON, MA, 02121","-71.07103,42.31112",85701960.0,R,25.0,25.0,90300.0,1000.0
1,17284,"271 NORMANDY ST, Boston, MA, 02121",Match,Exact,"271 NORMANDY ST, BOSTON, MA, 02121","-71.08389,42.30395",85701551.0,R,25.0,25.0,90100.0,1012.0
2,17285,"275 NORMANDY ST, Boston, MA, 02121",Match,Exact,"275 NORMANDY ST, BOSTON, MA, 02121","-71.083916,42.303905",85701551.0,R,25.0,25.0,90100.0,1012.0
3,17286,"279 NORMANDY ST, Boston, MA, 02121",Match,Exact,"279 NORMANDY ST, BOSTON, MA, 02121","-71.08393,42.30386",85701551.0,R,25.0,25.0,90100.0,1012.0
4,17287,"12 COLUMBIA RD, Boston, MA, 02121",Match,Exact,"12 COLUMBIA RD, BOSTON, MA, 02121","-71.08453,42.303528",85728832.0,L,25.0,25.0,90100.0,1012.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,12294,"85 CEDAR ST, Boston, MA, 02119",Match,Exact,"85 CEDAR ST, BOSTON, MA, 02119","-71.09216,42.32657",85699204.0,R,25.0,25.0,81400.0,1006.0
9995,12297,"82 HIGHLAND ST, Boston, MA, 02119",Match,Exact,"82 HIGHLAND ST, BOSTON, MA, 02119","-71.092674,42.327377",85699202.0,L,25.0,25.0,81400.0,1006.0
9996,12296,"91 CEDAR ST, Boston, MA, 02119",Match,Exact,"91 CEDAR ST, BOSTON, MA, 02119","-71.09243,42.326748",85699204.0,R,25.0,25.0,81400.0,1006.0
9997,12299,"15 DORR ST, Boston, MA, 02119",Match,Exact,"15 DORR ST, BOSTON, MA, 02119","-71.09216,42.327515",85699203.0,R,25.0,25.0,81400.0,1006.0


In [ ]:



df = pd.read_csv('AddressBook\GeocodeResults_1.csv');
df = pd.read_csv('AddressBook\GeocodeResults_2.csv');
df = pd.read_csv('AddressBook\GeocodeResults_3.csv');
df = pd.read_csv('AddressBook\GeocodeResults_4.csv');
df = pd.read_csv('AddressBook\GeocodeResults_5.csv');
df = pd.read_csv('AddressBook\GeocodeResults_6.csv');
df = pd.read_csv('AddressBook\GeocodeResults_snum.csv');

In [6]:
df = pd.read_csv('AddressBook\GeocodeResults_1.csv');

In [10]:
df.loc[[1,3,4]]

,4970,"70 FERRIN ST, Boston, MA, 02129",Match,Exact,"70 FERRIN ST, BOSTON, MA, 02129","-71.05843,42.376926",85709167,L,25,025,040200,2002
1,4971,"68 FERRIN ST, Boston, MA, 02129",Match,Exact,"68 FERRIN ST, BOSTON, MA, 02129","-71.05843,42.376923",85709167.0,L,25.0,25.0,40200.0,2002.0
3,3638,"210 SARATOGA ST, Boston, MA, 02128",Match,Exact,"210 SARATOGA ST, BOSTON, MA, 02128","-71.033745,42.37759",85711559.0,L,25.0,25.0,50200.0,3001.0
4,4969,"16 TREMONT ST, Boston, MA, 02129",Match,Exact,"16 TREMONT ST, BOSTON, MA, 02129","-71.05902,42.37648",85709169.0,L,25.0,25.0,40200.0,2002.0
